In [ ]:
import tecton
tc = tecton.TectonContext.create_new()
session_actiavte = tecton.login('ds_tester_6','DQwjfi!1uaqKIG9w')

In [ ]:
# tc.list_tables()

In [ ]:
# could not find a way to select table as data from context so had to run this query. I am not sure about execution plan if it would be optimal or not

# What is the point in keeping Data in S3 buckets as Tecton Data Frame if in the end we have to convert them to pandas or spark to do some work
order_products_train_df = tc.execute("""SELECT * FROM instacart.order_products__train""").to_spark()
order_products_prior_df = tc.execute("""SELECT * FROM instacart.order_products__prior""").to_spark()

orders_df = tc.execute("""SELECT * FROM instacart.orders""").to_spark()
products_df = tc.execute("""SELECT * FROM instacart.products""").to_spark()

aisles_df = tc.execute("""SELECT * FROM instacart.aisles""").to_spark()
departments_df = tc.execute("""SELECT * FROM instacart.departments""").to_spark()

In [ ]:
orders_df.head()

In [ ]:
## Joining all tables and and product train data

df_train = order_products_train_df.join(orders_df, order_products_train_df.order_id == orders_df.order_id, how='left')
df_train = df_train.join(products_df, df_train.product_id == products_df.product_id, how='left')
df_train = df_train.join(aisles_df, df_train.aisle_id == aisles_df.aisle_id, how='left')
df_train = df_train.join(departments_df, df_train.department_id == departments_df.department_id, how='left')

In [ ]:
df_train.limit(10).toPandas()

In [ ]:
## Joining all tables and and product prior data which contains data of all previous purchases

df_prior = order_products_prior_df.join(orders_df, order_products_prior_df.order_id == orders_df.order_id, how='left').drop(orders_df.order_id).limit(10000)
df_prior = df_prior.join(products_df, df_prior.product_id == products_df.product_id, how='left').drop(products_df.product_id)
df_prior = df_prior.join(aisles_df, df_prior.aisle_id == aisles_df.aisle_id, how='left').drop(aisles_df.aisle_id)
df_prior = df_prior.join(departments_df, df_prior.department_id == departments_df.department_id, how='left').drop(departments_df.department_id)

In [ ]:
df_prior.head()

In [ ]:
## Checking shape of products training table

df_train.toPandas().shape

In [ ]:
## Checking shape of products prior table

df_prior.toPandas().shape

In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate(SparkConf())
sqlContext = SQLContext(sc)

### Making User features¶

In [ ]:
## This table calculates unique products,departments,aisle and order for each customer. 
##It also calculates user order rate and other important feattures like days and hours since prev order


df_train.registerTempTable("df_train_tab")
df_prior.registerTempTable("df_prior_tab")
products_df.registerTempTable("products_df")
df_user_fund = sqlContext.sql("""SELECT user_id ,
count(1) as user_item_cnt,
COUNT(DISTINCT product_id) as user_prd_cnt,
  COUNT(DISTINCT department_id) as user_depart_cnt,
  COUNT(DISTINCT aisle_id) as user_aisle_cnt,
  COUNT(DISTINCT order_id) as user_order_cnt,
  COUNT(DISTINCT order_id) / count(1)  as user_order_rate,
  MAX(order_number) as max_order_number,
  AVG(days_since_prior_order) as avg_days_since_prior_order,
  MAX(days_since_prior_order) as max_days_since_prior_order,
  MIN(days_since_prior_order) as min_days_since_prior_order,
  MAX(order_hour_of_day) as max_order_hour_of_day,
  MIN(order_hour_of_day) as min_order_hour_of_day,
  AVG(order_hour_of_day) as avg_order_hour_of_day,
  AVG(reordered) as avg_reordered,
  SUM(reordered) as sum_reordered,
  AVG(order_dow) as avg_order_dow
               FROM df_prior_tab GROUP BY user_id""")

In [ ]:
## This table calculates max , min and avg days since each user has placed order

orders_df.registerTempTable("df_order_tab")
df_user_freq = sqlContext.sql("""SELECT
  user_id,
  count(1) as order_cnt,
  AVG(days_since_prior_order) as avg_days_since_prior_order,
  MAX(days_since_prior_order) as max_days_since_prior_order,
  MIN(days_since_prior_order) as min_days_since_prior_order
    FROM
      df_order_tab
    WHERE eval_set = 'prior'
    GROUP BY
      user_id""")

In [ ]:
## This calculates sum and avg of days when customer has ordered and re-ordered a product

df_user_dow = sqlContext.sql("""SELECT
  user_id,
  sum(CASE WHEN order_dow = 0  THEN 1 ELSE 0 END) AS  order_dow_0,
  sum(CASE WHEN order_dow = 1  THEN 1 ELSE 0 END) AS  order_dow_1,
  sum(CASE WHEN order_dow = 2  THEN 1 ELSE 0 END) AS  order_dow_2,
  sum(CASE WHEN order_dow = 3  THEN 1 ELSE 0 END) AS  order_dow_3,
  sum(CASE WHEN order_dow = 4  THEN 1 ELSE 0 END) AS  order_dow_4,
  sum(CASE WHEN order_dow = 5  THEN 1 ELSE 0 END) AS  order_dow_5,
  sum(CASE WHEN order_dow = 6  THEN 1 ELSE 0 END) AS  order_dow_6,

  avg(CASE WHEN order_dow = 0  THEN reordered ELSE null END) AS  reorder_dow_0,
  avg(CASE WHEN order_dow = 1  THEN reordered ELSE null END) AS  reorder_dow_1,
  avg(CASE WHEN order_dow = 2  THEN reordered ELSE null END) AS  reorder_dow_2,
  avg(CASE WHEN order_dow = 3  THEN reordered ELSE null END) AS  reorder_dow_3,
  avg(CASE WHEN order_dow = 4  THEN reordered ELSE null END) AS  reorder_dow_4,
  avg(CASE WHEN order_dow = 5  THEN reordered ELSE null END) AS  reorder_dow_5,
  avg(CASE WHEN order_dow = 6  THEN reordered ELSE null END) AS  reorder_dow_6
FROM
  df_prior_tab
GROUP BY
  user_id""")

In [ ]:
## This calculates sum of hours of days for each customer when he has ordered a product

df_user_hour = sqlContext.sql("""SELECT
  user_id,
  sum(CASE WHEN order_hour_of_day = 0  THEN 1 ELSE 0 END) AS order_hour_of_day_0,
  sum(CASE WHEN order_hour_of_day = 1  THEN 1 ELSE 0 END) AS order_hour_of_day_1,
  sum(CASE WHEN order_hour_of_day = 2  THEN 1 ELSE 0 END) AS order_hour_of_day_2,
  sum(CASE WHEN order_hour_of_day = 3  THEN 1 ELSE 0 END) AS order_hour_of_day_3,
  sum(CASE WHEN order_hour_of_day = 4  THEN 1 ELSE 0 END) AS order_hour_of_day_4,
  sum(CASE WHEN order_hour_of_day = 5  THEN 1 ELSE 0 END) AS order_hour_of_day_5,
  sum(CASE WHEN order_hour_of_day = 6  THEN 1 ELSE 0 END) AS order_hour_of_day_6,
  sum(CASE WHEN order_hour_of_day = 7  THEN 1 ELSE 0 END) AS order_hour_of_day_7,
  sum(CASE WHEN order_hour_of_day = 8  THEN 1 ELSE 0 END) AS order_hour_of_day_8,
  sum(CASE WHEN order_hour_of_day = 9  THEN 1 ELSE 0 END) AS order_hour_of_day_9,
  sum(CASE WHEN order_hour_of_day = 10  THEN 1 ELSE 0 END) AS order_hour_of_day_10,
  sum(CASE WHEN order_hour_of_day = 11  THEN 1 ELSE 0 END) AS order_hour_of_day_11,
  sum(CASE WHEN order_hour_of_day = 12  THEN 1 ELSE 0 END) AS order_hour_of_day_12,
  sum(CASE WHEN order_hour_of_day = 13  THEN 1 ELSE 0 END) AS order_hour_of_day_13,
  sum(CASE WHEN order_hour_of_day = 14  THEN 1 ELSE 0 END) AS order_hour_of_day_14,
  sum(CASE WHEN order_hour_of_day = 15  THEN 1 ELSE 0 END) AS order_hour_of_day_15,
  sum(CASE WHEN order_hour_of_day = 16  THEN 1 ELSE 0 END) AS order_hour_of_day_16,
  sum(CASE WHEN order_hour_of_day = 17  THEN 1 ELSE 0 END) AS order_hour_of_day_17,
  sum(CASE WHEN order_hour_of_day = 18  THEN 1 ELSE 0 END) AS order_hour_of_day_18,
  sum(CASE WHEN order_hour_of_day = 19  THEN 1 ELSE 0 END) AS order_hour_of_day_19,
  sum(CASE WHEN order_hour_of_day = 20  THEN 1 ELSE 0 END) AS order_hour_of_day_20,
  sum(CASE WHEN order_hour_of_day = 21  THEN 1 ELSE 0 END) AS order_hour_of_day_21,
  sum(CASE WHEN order_hour_of_day = 22  THEN 1 ELSE 0 END) AS order_hour_of_day_22,
  sum(CASE WHEN order_hour_of_day = 23  THEN 1 ELSE 0 END) AS order_hour_of_day_23
FROM
  df_prior_tab
GROUP BY
  user_id""")

In [ ]:
## This calculates times a customer has ordered from same department


df_user_depart = sqlContext.sql("""SELECT
  user_id,
  sum(CASE WHEN department_id = 1  THEN 1 ELSE 0 END) AS department_id_1,
  sum(CASE WHEN department_id = 2  THEN 1 ELSE 0 END) AS department_id_2,
  sum(CASE WHEN department_id = 3  THEN 1 ELSE 0 END) AS department_id_3,
  sum(CASE WHEN department_id = 4  THEN 1 ELSE 0 END) AS department_id_4,
  sum(CASE WHEN department_id = 5  THEN 1 ELSE 0 END) AS department_id_5,
  sum(CASE WHEN department_id = 6  THEN 1 ELSE 0 END) AS department_id_6,
  sum(CASE WHEN department_id = 7  THEN 1 ELSE 0 END) AS department_id_7,
  sum(CASE WHEN department_id = 8  THEN 1 ELSE 0 END) AS department_id_8,
  sum(CASE WHEN department_id = 9  THEN 1 ELSE 0 END) AS department_id_9,
  sum(CASE WHEN department_id = 10  THEN 1 ELSE 0 END) AS department_id_10,
  sum(CASE WHEN department_id = 11  THEN 1 ELSE 0 END) AS department_id_11,
  sum(CASE WHEN department_id = 12  THEN 1 ELSE 0 END) AS department_id_12,
  sum(CASE WHEN department_id = 13  THEN 1 ELSE 0 END) AS department_id_13,
  sum(CASE WHEN department_id = 14  THEN 1 ELSE 0 END) AS department_id_14,
  sum(CASE WHEN department_id = 15  THEN 1 ELSE 0 END) AS department_id_15,
  sum(CASE WHEN department_id = 16  THEN 1 ELSE 0 END) AS department_id_16,
  sum(CASE WHEN department_id = 17  THEN 1 ELSE 0 END) AS department_id_17,
  sum(CASE WHEN department_id = 18  THEN 1 ELSE 0 END) AS department_id_18,
  sum(CASE WHEN department_id = 19  THEN 1 ELSE 0 END) AS department_id_19,
  sum(CASE WHEN department_id = 20  THEN 1 ELSE 0 END) AS department_id_20,
  sum(CASE WHEN department_id = 21  THEN 1 ELSE 0 END) AS department_id_21
FROM
  df_prior_tab
GROUP BY
  user_id""")

In [ ]:
## This table merges all the data collected in upper tables in single table

df_user_fund.registerTempTable("df_user_fund")
df_user_dow.registerTempTable("df_user_dow")
df_user_hour.registerTempTable("df_user_hour")
df_user_depart.registerTempTable("df_user_depart")
df_user_freq.registerTempTable("df_user_freq")

df_dmt_user = sqlContext.sql("""SELECT u1.user_id,
 user_item_cnt,
 user_prd_cnt,
 user_depart_cnt,
 user_aisle_cnt,
 user_order_cnt,
 user_order_rate,
 max_order_number,
 u1.avg_days_since_prior_order,
 u1.max_days_since_prior_order,
 u1.min_days_since_prior_order,
 max_order_hour_of_day,
 min_order_hour_of_day,
 avg_order_hour_of_day,
 avg_reordered,
 sum_reordered,
 avg_order_dow,
 order_dow_0,
 order_dow_1,
 order_dow_2,
 order_dow_3,
 order_dow_4,
 order_dow_5,
 order_dow_6,
 reorder_dow_0,
 reorder_dow_1,
 reorder_dow_2,
 reorder_dow_3,
 reorder_dow_4,
 reorder_dow_5,
 reorder_dow_6,
 order_hour_of_day_0,
 order_hour_of_day_1,
 order_hour_of_day_2,
 order_hour_of_day_3,
 order_hour_of_day_4,
 order_hour_of_day_5,
 order_hour_of_day_6,
 order_hour_of_day_7,
 order_hour_of_day_8,
 order_hour_of_day_9,
 order_hour_of_day_10,
 order_hour_of_day_11,
 order_hour_of_day_12,
 order_hour_of_day_13,
 order_hour_of_day_14,
 order_hour_of_day_15,
 order_hour_of_day_16,
 order_hour_of_day_17,
 order_hour_of_day_18,
 order_hour_of_day_19,
 order_hour_of_day_20,
 order_hour_of_day_21,
 order_hour_of_day_22,
 order_hour_of_day_23,
 department_id_1,
 department_id_2,
 department_id_3,
 department_id_4,
 department_id_5,
 department_id_6,
 department_id_7,
 department_id_8,
 department_id_9,
 department_id_10,
 department_id_11,
 department_id_12,
 department_id_13,
 department_id_14,
 department_id_15,
 department_id_16,
 department_id_17,
 department_id_18,
 department_id_19,
 department_id_20,
 department_id_21,
 order_cnt
FROM
  df_user_fund as u1
LEFT OUTER JOIN
  df_user_dow as u2
ON
  u1.user_id = u2.user_id
LEFT OUTER JOIN
  df_user_hour as u3
ON
  u1.user_id = u3.user_id
LEFT OUTER JOIN
  df_user_depart as u4
ON
  u1.user_id = u4.user_id
LEFT OUTER JOIN
  df_user_freq as u5
ON
  u1.user_id = u5.user_id""")
df_dmt_user.registerTempTable("df_dmt_user")

### Making item features¶


In [ ]:
## This table calculates unique user,departments,aisle and order for each product. 
##It also calculates product order rate and other important feattures like days and hours since prev order


df_dmt_user.registerTempTable("df_dmt_user")

df_item_fund = sqlContext.sql("""SELECT
  product_id,
  count(1) as item_user_cnt,
  COUNT(DISTINCT user_id) as item_usr_cnt,
  COUNT(DISTINCT department_id) as item_depart_cnt,
  COUNT(DISTINCT aisle_id) as item_aisle_cnt,
  COUNT(DISTINCT order_id) as item_order_cnt,
  COUNT(DISTINCT order_id) / count(1) as item_order_rate,
  AVG(days_since_prior_order) as avg_item_days_since_prior_order,
  MIN(days_since_prior_order) as min_item_days_since_prior_order,
  MAX(days_since_prior_order) as max_item_days_since_prior_order,
  MAX(order_hour_of_day) as max_order_hour_of_day,
  MIN(order_hour_of_day) as min_order_hour_of_day,
  AVG(order_hour_of_day) as avg_order_hour_of_day,
  AVG(reordered) as avg_item_reordered,
  SUM(reordered) as sum_item_reordered,
  AVG(order_dow) as avg_order_dow
FROM
  df_prior_tab
GROUP BY
  product_id""")

In [ ]:
## This calculates sum of days when procuct has been ordered  

df_item_dow = sqlContext.sql("""SELECT
  product_id,
  sum(CASE WHEN order_dow = 0  THEN 1 ELSE 0 END) AS  order_dow_0,
  sum(CASE WHEN order_dow = 1  THEN 1 ELSE 0 END) AS  order_dow_1,
  sum(CASE WHEN order_dow = 2  THEN 1 ELSE 0 END) AS  order_dow_2,
  sum(CASE WHEN order_dow = 3  THEN 1 ELSE 0 END) AS  order_dow_3,
  sum(CASE WHEN order_dow = 4  THEN 1 ELSE 0 END) AS  order_dow_4,
  sum(CASE WHEN order_dow = 5  THEN 1 ELSE 0 END) AS  order_dow_5,
  sum(CASE WHEN order_dow = 6  THEN 1 ELSE 0 END) AS  order_dow_6
FROM
  df_prior_tab
GROUP BY
  product_id""")

In [ ]:
## This calculates sum of hours of days for each product ordered

df_item_hour = sqlContext.sql("""SELECT
  product_id,
  sum(CASE WHEN order_hour_of_day = 0  THEN 1 ELSE 0 END) AS order_hour_of_day_0,
  sum(CASE WHEN order_hour_of_day = 1  THEN 1 ELSE 0 END) AS order_hour_of_day_1,
  sum(CASE WHEN order_hour_of_day = 2  THEN 1 ELSE 0 END) AS order_hour_of_day_2,
  sum(CASE WHEN order_hour_of_day = 3  THEN 1 ELSE 0 END) AS order_hour_of_day_3,
  sum(CASE WHEN order_hour_of_day = 4  THEN 1 ELSE 0 END) AS order_hour_of_day_4,
  sum(CASE WHEN order_hour_of_day = 5  THEN 1 ELSE 0 END) AS order_hour_of_day_5,
  sum(CASE WHEN order_hour_of_day = 6  THEN 1 ELSE 0 END) AS order_hour_of_day_6,
  sum(CASE WHEN order_hour_of_day = 7  THEN 1 ELSE 0 END) AS order_hour_of_day_7,
  sum(CASE WHEN order_hour_of_day = 8  THEN 1 ELSE 0 END) AS order_hour_of_day_8,
  sum(CASE WHEN order_hour_of_day = 9  THEN 1 ELSE 0 END) AS order_hour_of_day_9,
  sum(CASE WHEN order_hour_of_day = 10  THEN 1 ELSE 0 END) AS order_hour_of_day_10,
  sum(CASE WHEN order_hour_of_day = 11  THEN 1 ELSE 0 END) AS order_hour_of_day_11,
  sum(CASE WHEN order_hour_of_day = 12  THEN 1 ELSE 0 END) AS order_hour_of_day_12,
  sum(CASE WHEN order_hour_of_day = 13  THEN 1 ELSE 0 END) AS order_hour_of_day_13,
  sum(CASE WHEN order_hour_of_day = 14  THEN 1 ELSE 0 END) AS order_hour_of_day_14,
  sum(CASE WHEN order_hour_of_day = 15  THEN 1 ELSE 0 END) AS order_hour_of_day_15,
  sum(CASE WHEN order_hour_of_day = 16  THEN 1 ELSE 0 END) AS order_hour_of_day_16,
  sum(CASE WHEN order_hour_of_day = 17  THEN 1 ELSE 0 END) AS order_hour_of_day_17,
  sum(CASE WHEN order_hour_of_day = 18  THEN 1 ELSE 0 END) AS order_hour_of_day_18,
  sum(CASE WHEN order_hour_of_day = 19  THEN 1 ELSE 0 END) AS order_hour_of_day_19,
  sum(CASE WHEN order_hour_of_day = 20  THEN 1 ELSE 0 END) AS order_hour_of_day_20,
  sum(CASE WHEN order_hour_of_day = 21  THEN 1 ELSE 0 END) AS order_hour_of_day_21,
  sum(CASE WHEN order_hour_of_day = 22  THEN 1 ELSE 0 END) AS order_hour_of_day_22,
  sum(CASE WHEN order_hour_of_day = 23  THEN 1 ELSE 0 END) AS order_hour_of_day_23
FROM
  df_prior_tab
GROUP BY
  product_id""")

In [ ]:
## This calculates times a product has been ordered from same department


df_item_depart = sqlContext.sql("""SELECT
  product_id,
  sum(CASE WHEN department_id = 1  THEN 1 ELSE 0 END) AS department_id_1,
  sum(CASE WHEN department_id = 2  THEN 1 ELSE 0 END) AS department_id_2,
  sum(CASE WHEN department_id = 3  THEN 1 ELSE 0 END) AS department_id_3,
  sum(CASE WHEN department_id = 4  THEN 1 ELSE 0 END) AS department_id_4,
  sum(CASE WHEN department_id = 5  THEN 1 ELSE 0 END) AS department_id_5,
  sum(CASE WHEN department_id = 6  THEN 1 ELSE 0 END) AS department_id_6,
  sum(CASE WHEN department_id = 7  THEN 1 ELSE 0 END) AS department_id_7,
  sum(CASE WHEN department_id = 8  THEN 1 ELSE 0 END) AS department_id_8,
  sum(CASE WHEN department_id = 9  THEN 1 ELSE 0 END) AS department_id_9,
  sum(CASE WHEN department_id = 10  THEN 1 ELSE 0 END) AS department_id_10,
  sum(CASE WHEN department_id = 11  THEN 1 ELSE 0 END) AS department_id_11,
  sum(CASE WHEN department_id = 12  THEN 1 ELSE 0 END) AS department_id_12,
  sum(CASE WHEN department_id = 13  THEN 1 ELSE 0 END) AS department_id_13,
  sum(CASE WHEN department_id = 14  THEN 1 ELSE 0 END) AS department_id_14,
  sum(CASE WHEN department_id = 15  THEN 1 ELSE 0 END) AS department_id_15,
  sum(CASE WHEN department_id = 16  THEN 1 ELSE 0 END) AS department_id_16,
  sum(CASE WHEN department_id = 17  THEN 1 ELSE 0 END) AS department_id_17,
  sum(CASE WHEN department_id = 18  THEN 1 ELSE 0 END) AS department_id_18,
  sum(CASE WHEN department_id = 19  THEN 1 ELSE 0 END) AS department_id_19,
  sum(CASE WHEN department_id = 20  THEN 1 ELSE 0 END) AS department_id_20,
  sum(CASE WHEN department_id = 21  THEN 1 ELSE 0 END) AS department_id_21
FROM
  df_prior_tab
GROUP BY
  product_id""")

In [ ]:
## This table merges all the data collected in upper tables in single table


df_item_fund.registerTempTable("df_item_fund")
df_item_dow.registerTempTable("df_item_dow")
df_item_hour.registerTempTable("df_item_hour")
df_item_depart.registerTempTable("df_item_depart")

df_dmt_item = sqlContext.sql("""SELECT
  i1.product_id,
 item_user_cnt,
 item_usr_cnt,
 item_depart_cnt,
 item_aisle_cnt,
 item_order_cnt,
 item_order_rate,
 avg_item_days_since_prior_order,
 min_item_days_since_prior_order,
 max_item_days_since_prior_order,
 max_order_hour_of_day,
 min_order_hour_of_day,
 avg_order_hour_of_day,
 avg_item_reordered,
 sum_item_reordered,
 avg_order_dow,
 order_dow_0,
 order_dow_1,
 order_dow_2,
 order_dow_3,
 order_dow_4,
 order_dow_5,
 order_dow_6,
 order_hour_of_day_0,
 order_hour_of_day_1,
 order_hour_of_day_2,
 order_hour_of_day_3,
 order_hour_of_day_4,
 order_hour_of_day_5,
 order_hour_of_day_6,
 order_hour_of_day_7,
 order_hour_of_day_8,
 order_hour_of_day_9,
 order_hour_of_day_10,
 order_hour_of_day_11,
 order_hour_of_day_12,
 order_hour_of_day_13,
 order_hour_of_day_14,
 order_hour_of_day_15,
 order_hour_of_day_16,
 order_hour_of_day_17,
 order_hour_of_day_18
 order_hour_of_day_19,
 order_hour_of_day_20,
 order_hour_of_day_21,
 order_hour_of_day_22,
 order_hour_of_day_23,
 department_id_1,
 department_id_2,
 department_id_3,
 department_id_4,
 department_id_5,
 department_id_6,
 department_id_7,
 department_id_8,
 department_id_9,
 department_id_10,
 department_id_11,
 department_id_12,
 department_id_13,
 department_id_14,
 department_id_15,
 department_id_16,
 department_id_17,
 department_id_18,
 department_id_19,
 department_id_20,
 department_id_21
FROM
  df_item_fund as i1
LEFT OUTER JOIN
  df_item_dow as i2
ON
  i1.product_id = i2.product_id
LEFT OUTER JOIN
  df_item_hour as i3
ON
  i1.product_id = i3.product_id
LEFT OUTER JOIN
  df_item_depart as i4
ON
  i1.product_id = i4.product_id""")
df_dmt_item.registerTempTable("df_dmt_item")

### Making user and item features

In [ ]:
## This table gets main feature from order table

df_work_tmp1 = sqlContext.sql("""SELECT
  user_id,
  order_id,
  eval_set,
  order_number,
  order_dow,
  order_hour_of_day,  
  days_since_prior_order,
  SUM(days_since_prior_order) OVER (PARTITION BY user_id ORDER BY order_number ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_days
FROM
  df_order_tab""")

In [ ]:
##This table adds another feature to prev table that is days since order has been placed by user

df_work_tmp1.registerTempTable("df_work_tmp1")
df_cum_orders = sqlContext.sql("""SELECT
  a.user_id user_id,
  a.order_id order_id,
  a.eval_set eval_set,
  a.order_number order_number,
  a.days_since_prior_order days_since_prior_order,
  a.cum_days cum_days,
  a.order_dow,
  a.order_hour_of_day,  
  b.max_cum_days max_cum_days,
  b.max_cum_days - a.cum_days as last_buy
FROM
  df_work_tmp1 as a
LEFT OUTER JOIN
(
SELECT
  user_id, max(cum_days) as max_cum_days
FROM
  df_work_tmp1
GROUP BY
  user_id
) as b
ON
  a.user_id = b.user_id""")

In [ ]:
## This table groups users and bought products together

df_only_rebuy = sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.product_id as product_id
FROM
  df_prior_tab as a
GROUP BY
  user_id, product_id""")

In [ ]:
## making training dataset using rebuy,orders and cum_order table

df_only_rebuy.registerTempTable("df_only_rebuy")
df_cum_orders.registerTempTable("df_cum_orders")


df_only_rebuy_train = sqlContext.sql("""SELECT
  r.user_id as user_id,
  r.product_id as product_id,
  o.order_id as order_id,
  o.order_number as order_number,
  o.order_dow as order_dow,
  o.order_hour_of_day as order_hour_of_day,
  o.days_since_prior_order as days_since_prior_order,
  c.cum_days as cum_days
FROM
  df_only_rebuy as r
INNER JOIN
  (SELECT * FROM df_order_tab WHERE eval_set='train') as o
ON
  r.user_id = o.user_id
LEFT OUTER JOIN
  df_cum_orders as c
ON
  c.order_id = o.order_id""")

In [ ]:
## making testing dataset using rebuy,orders and cum_order table

df_only_rebuy_test = sqlContext.sql("""SELECT
  r.user_id as user_id,
  r.product_id as product_id,
  o.order_id as order_id,
  o.order_number as order_number,
  o.order_dow as order_dow,
  o.order_hour_of_day as order_hour_of_day,
  o.days_since_prior_order as days_since_prior_order,
  c.cum_days as cum_days
FROM
  df_only_rebuy as r
INNER JOIN
  (SELECT * FROM df_order_tab WHERE eval_set='test') as o
ON
  r.user_id = o.user_id
LEFT OUTER JOIN
  df_cum_orders as c
ON
  c.order_id = o.order_id""")

In [ ]:
## This table creates a table to get info about last bought items against a user

df_last_buy = sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  a.max_order_number as max_order_number,
  a.max_reordered as max_reordered,
  a.avg_reordered as avg_reordered,
  o.order_id,
  c.cum_days as cum_days,
  c.last_buy as last_buy,
  o.order_number,
  o.order_dow,
  o.order_hour_of_day,
  o.days_since_prior_order,
  o.order_number - c.order_number as order_number_diff,
  o.days_since_prior_order - c.days_since_prior_order
FROM
  (
  SELECT
    user_id,
    product_id,
    max(reordered) as max_reordered,
    avg(reordered) as avg_reordered,
    MAX(order_number) as max_order_number
  FROM
    df_prior_tab
  GROUP BY
    user_id,
    product_id
  ) as a
LEFT OUTER JOIN
  df_order_tab as o
ON
  a.user_id = o.user_id AND
  a.max_order_number = o.order_number
LEFT OUTER JOIN
  df_cum_orders as c
ON
  c.order_id = o.order_id""")

In [ ]:
## This table creates a table to get info about last bought items by a user using aisle id 


df_last_buy_aisle = sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.aisle_id as aisle_id,
  a.max_order_number as max_order_number,
  a.max_reordered as max_reordered,
  a.avg_reordered as avg_reordered,
  o.order_id,
  c.cum_days as cum_days,
  c.last_buy as last_buy,
  o.order_number,
  o.order_dow,
  o.order_hour_of_day,
  o.days_since_prior_order,
  o.order_number - c.order_number as order_number_diff,
  o.days_since_prior_order - c.days_since_prior_order
FROM
  (
  SELECT
    user_id,
    aisle_id,
    max(reordered) as max_reordered,
    avg(reordered) as avg_reordered,
    MAX(order_number) as max_order_number
  FROM
    df_prior_tab
  GROUP BY
    user_id,
    aisle_id
  ) as a
LEFT OUTER JOIN
  df_order_tab as o
ON
  a.user_id = o.user_id AND
  a.max_order_number = o.order_number
LEFT OUTER JOIN
  df_cum_orders as c
ON
  c.order_id = o.order_id""")

In [ ]:
## This table creates a table to get info about last bought items by a user in context with department 

df_last_buy_depart= sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.department_id as department_id,
  a.max_order_number as max_order_number,
  a.max_reordered as max_reordered,
  a.avg_reordered as avg_reordered,
  o.order_id,
  c.cum_days as cum_days,
  c.last_buy as last_buy,
  o.order_number,
  o.order_dow,
  o.order_hour_of_day,
  o.days_since_prior_order,
  o.order_number - c.order_number as order_number_diff,
  o.days_since_prior_order - c.days_since_prior_order
FROM
  (
  SELECT
    user_id,
    department_id,
    max(reordered) as max_reordered,
    avg(reordered) as avg_reordered,
    MAX(order_number) as max_order_number
  FROM
    df_prior_tab
  GROUP BY
    user_id,
    department_id
  ) as a
LEFT OUTER JOIN
  df_order_tab as o
ON
  a.user_id = o.user_id AND
  a.max_order_number = o.order_number
LEFT OUTER JOIN
  df_cum_orders as c
ON
  c.order_id = o.order_id""")

In [ ]:
## This table creates a table to get info about last bought items against a user with 2nd max order


df_last_buy_2= sqlContext.sql("""SELECT 
  a.user_id as user_id,
  a.product_id as product_id,
  a.max_order_number as max_order_number,
  a.max_reordered as max_reordered,
  a.avg_reordered as avg_reordered,
  o.order_id,
  c.cum_days as cum_days,
  c.last_buy as last_buy,
  o.order_number,
  o.order_dow,
  o.order_hour_of_day,
  o.days_since_prior_order,
  o.order_number - c.order_number as order_number_diff,
  o.days_since_prior_order - c.days_since_prior_order
FROM
  (
  SELECT
    user_id,
    product_id,
    max(reordered) as max_reordered,
    avg(reordered) as avg_reordered,
    MAX(order_number) - 1 as max_order_number
  FROM
    df_prior_tab
  GROUP BY
    user_id,
    product_id
  ) as a
LEFT OUTER JOIN
  df_order_tab as o
ON
  a.user_id = o.user_id AND
  a.max_order_number = o.order_number
LEFT OUTER JOIN
  df_cum_orders as c
ON
  c.order_id = o.order_id""")

In [ ]:
## This table creates a table to get info about 2nd last bought items by a user using aisle id 


df_last_buy_aisle_2= sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.aisle_id as aisle_id,
  a.max_order_number as max_order_number,
  a.max_reordered as max_reordered,
  a.avg_reordered as avg_reordered,
  o.order_id,
  c.cum_days as cum_days,
  c.last_buy as last_buy,
  o.order_number,
  o.order_dow,
  o.order_hour_of_day,
  o.days_since_prior_order,
  o.order_number - c.order_number as order_number_diff,
  o.days_since_prior_order - c.days_since_prior_order
FROM
  (
  SELECT
    user_id,
    aisle_id,
    max(reordered) as max_reordered,
    avg(reordered) as avg_reordered,
    MAX(order_number) - 1 as max_order_number
  FROM
    df_prior_tab
  GROUP BY
    user_id,
    aisle_id
  ) as a
LEFT OUTER JOIN
  df_order_tab as o
ON
  a.user_id = o.user_id AND
  a.max_order_number = o.order_number
LEFT OUTER JOIN
  df_cum_orders as c
ON
  c.order_id = o.order_id""")

In [ ]:
## This table creates a table to get info about 2nd last bought items by a user in context with department 


df_last_buy_depart_2= sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.department_id as department_id,
  a.max_order_number as max_order_number,
  a.max_reordered as max_reordered,
  a.avg_reordered as avg_reordered,
  o.order_id,
  c.cum_days as cum_days,
  c.last_buy as last_buy,
  o.order_number,
  o.order_dow,
  o.order_hour_of_day,
  o.days_since_prior_order,
  o.order_number - c.order_number as order_number_diff,
  o.days_since_prior_order - c.days_since_prior_order
FROM
  (
  SELECT
    user_id,
    department_id,
    max(reordered) as max_reordered,
    avg(reordered) as avg_reordered,
    MAX(order_number) - 1 as max_order_number
  FROM
    df_prior_tab
  GROUP BY
    user_id,
    department_id
  ) as a
LEFT OUTER JOIN
  df_order_tab as o
ON
  a.user_id = o.user_id AND
  a.max_order_number = o.order_number
LEFT OUTER JOIN
  df_cum_orders as c
ON
  c.order_id = o.order_id""")

In [ ]:
## This table makes features for user bought products for last month

df_dmt_user_item_30= sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  count(1) cnt_user_item,
  COUNT(DISTINCT a.order_id) cnt_user_order,
  avg(a.order_hour_of_day) avg_order_hour_of_day,
  min(a.order_hour_of_day) min_order_hour_of_day,
  max(a.order_hour_of_day) max_order_hour_of_day,
  max(a.reordered) max_reordered,
  sum(a.reordered) sum_reordered,
  avg(a.reordered) avg_reordered,
  AVG(a.order_dow) as avg_order_dow,
  MIN(a.order_dow) as min_order_dow,
  MAX(a.order_dow) as max_order_dow,
  AVG(a.days_since_prior_order) as avg_days_since_prior_order,
  MAX(a.days_since_prior_order) as max_days_since_prior_order,
  MIN(a.days_since_prior_order) as min_days_since_prior_order,
  sum(CASE WHEN a.order_dow = 0  THEN 1 ELSE 0 END) AS  order_dow_0,
  sum(CASE WHEN a.order_dow = 1  THEN 1 ELSE 0 END) AS  order_dow_1,
  sum(CASE WHEN a.order_dow = 2  THEN 1 ELSE 0 END) AS  order_dow_2,
  sum(CASE WHEN a.order_dow = 3  THEN 1 ELSE 0 END) AS  order_dow_3,
  sum(CASE WHEN a.order_dow = 4  THEN 1 ELSE 0 END) AS  order_dow_4,
  sum(CASE WHEN a.order_dow = 5  THEN 1 ELSE 0 END) AS  order_dow_5,
  sum(CASE WHEN a.order_dow = 6  THEN 1 ELSE 0 END) AS  order_dow_6,
  avg(CASE WHEN a.order_dow = 0  THEN a.reordered ELSE null END) AS  reorder_dow_0,
  avg(CASE WHEN a.order_dow = 1  THEN a.reordered ELSE null END) AS  reorder_dow_1,
  avg(CASE WHEN a.order_dow = 2  THEN a.reordered ELSE null END) AS  reorder_dow_2,
  avg(CASE WHEN a.order_dow = 3  THEN a.reordered ELSE null END) AS  reorder_dow_3,
  avg(CASE WHEN a.order_dow = 4  THEN a.reordered ELSE null END) AS  reorder_dow_4,
  avg(CASE WHEN a.order_dow = 5  THEN a.reordered ELSE null END) AS  reorder_dow_5,
  avg(CASE WHEN a.order_dow = 6  THEN a.reordered ELSE null END) AS  reorder_dow_6
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
GROUP BY
  a.user_id, product_id""")


In [ ]:
## This table makes features for user bought products for last month in consideration with aisle

df_dmt_user_aisle_30= sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.aisle_id as aisle_id,
  count(1) cnt_user_aisle,
  COUNT(DISTINCT a.order_id) cnt_aisle_order,
  COUNT(DISTINCT a.product_id) cnt_product_order,
  avg(a.order_hour_of_day) avg_order_hour_of_day,
  min(a.order_hour_of_day) min_order_hour_of_day,
  max(a.order_hour_of_day) max_order_hour_of_day,
  AVG(a.days_since_prior_order) as avg_days_since_prior_order,
  MAX(a.days_since_prior_order) as max_days_since_prior_order,
  MIN(a.days_since_prior_order) as min_days_since_prior_order,
  AVG(a.order_dow) as avg_order_dow,
  MAX(a.order_dow) as max_order_dow,
  MIN(a.order_dow) as min_order_dow,
  max(reordered) max_reordered,
  sum(reordered) sum_reordered,
  avg(reordered) avg_reordered
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
GROUP BY
  a.user_id, aisle_id""")

In [ ]:
## This table makes features for user bought products for last month in consideration with department


df_dmt_user_depart_30= sqlContext.sql("""SELECT
  a.user_id user_id,
  a.department_id department_id,
  count(1) cnt_user_depart,
  COUNT(DISTINCT a.order_id) cnt_depart_order,
  COUNT(DISTINCT a.product_id) cnt_product_order,
  COUNT(DISTINCT a.aisle_id) cnt_aisle_order,
  avg(a.order_hour_of_day) avg_order_hour_of_day,
  min(a.order_hour_of_day) min_order_hour_of_day,
  max(a.order_hour_of_day) max_order_hour_of_day,
  AVG(a.days_since_prior_order) as avg_days_since_prior_order,
  MAX(a.days_since_prior_order) as max_days_since_prior_order,
  MIN(a.days_since_prior_order) as min_days_since_prior_order,
  AVG(a.order_dow) as avg_order_dow,
  MAX(a.order_dow) as max_order_dow,
  MIN(a.order_dow) as min_order_dow,
  max(reordered) max_reordered,
  sum(reordered) sum_reordered,
  avg(reordered) avg_reordered
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
GROUP BY
  a.user_id, department_id""")

In [ ]:
## This table makes features for user bought products for last month


df_dmt_user_item= sqlContext.sql("""SELECT
  user_id,
  product_id,
  count(1) cnt_user_item,
  COUNT(DISTINCT order_id) cnt_user_order,
  avg(order_hour_of_day) avg_order_hour_of_day,
  min(order_hour_of_day) min_order_hour_of_day,
  max(order_hour_of_day) max_order_hour_of_day,
  max(reordered) max_reordered,
  sum(reordered) sum_reordered,
  avg(reordered) avg_reordered,
  AVG(days_since_prior_order) as avg_days_since_prior_order,
  MAX(days_since_prior_order) as max_days_since_prior_order,
  MIN(days_since_prior_order) as min_days_since_prior_order,
  AVG(order_dow) as avg_order_dow,
  MAX(order_dow) as max_order_dow,
  MIN(order_dow) as min_order_dow,
  sum(CASE WHEN order_dow = 0  THEN 1 ELSE 0 END) AS  order_dow_0,
  sum(CASE WHEN order_dow = 1  THEN 1 ELSE 0 END) AS  order_dow_1,
  sum(CASE WHEN order_dow = 2  THEN 1 ELSE 0 END) AS  order_dow_2,
  sum(CASE WHEN order_dow = 3  THEN 1 ELSE 0 END) AS  order_dow_3,
  sum(CASE WHEN order_dow = 4  THEN 1 ELSE 0 END) AS  order_dow_4,
  sum(CASE WHEN order_dow = 5  THEN 1 ELSE 0 END) AS  order_dow_5,
  sum(CASE WHEN order_dow = 6  THEN 1 ELSE 0 END) AS  order_dow_6,
  avg(CASE WHEN order_dow = 0  THEN reordered ELSE null END) AS  reorder_dow_0,
  avg(CASE WHEN order_dow = 1  THEN reordered ELSE null END) AS  reorder_dow_1,
  avg(CASE WHEN order_dow = 2  THEN reordered ELSE null END) AS  reorder_dow_2,
  avg(CASE WHEN order_dow = 3  THEN reordered ELSE null END) AS  reorder_dow_3,
  avg(CASE WHEN order_dow = 4  THEN reordered ELSE null END) AS  reorder_dow_4,
  avg(CASE WHEN order_dow = 5  THEN reordered ELSE null END) AS  reorder_dow_5,
  avg(CASE WHEN order_dow = 6  THEN reordered ELSE null END) AS  reorder_dow_6
FROM
  df_prior_tab
GROUP BY
  user_id, product_id""")

In [ ]:
## This table makes features for user bought products in consideration with aisle


df_dmt_user_aisle= sqlContext.sql("""SELECT
  user_id,
  aisle_id,
  count(1) cnt_user_aisle,
  COUNT(DISTINCT order_id) cnt_aisle_order,
  COUNT(DISTINCT product_id) cnt_product_order,
  avg(order_hour_of_day) avg_order_hour_of_day,
  min(order_hour_of_day) min_order_hour_of_day,
  max(order_hour_of_day) max_order_hour_of_day,
  AVG(days_since_prior_order) as avg_days_since_prior_order,
  MAX(days_since_prior_order) as max_days_since_prior_order,
  MIN(days_since_prior_order) as min_days_since_prior_order,
  AVG(order_dow) as avg_order_dow,
  MAX(order_dow) as max_order_dow,
  MIN(order_dow) as min_order_dow,
  max(reordered) max_reordered,
  sum(reordered) sum_reordered,
  avg(reordered) avg_reordered
FROM
  df_prior_tab
GROUP BY
  user_id, aisle_id""")

In [ ]:
## This table makes features for user bought products in consideration with department


df_dmt_user_depart= sqlContext.sql("""SELECT
  user_id,
  department_id,
  count(1) cnt_user_depart,
  COUNT(DISTINCT order_id) cnt_depart_order,
  COUNT(DISTINCT product_id) cnt_product_order,
  COUNT(DISTINCT aisle_id) cnt_aisle_order,
  avg(order_hour_of_day) avg_order_hour_of_day,
  min(order_hour_of_day) min_order_hour_of_day,
  max(order_hour_of_day) max_order_hour_of_day,
  AVG(days_since_prior_order) as avg_days_since_prior_order,
  MAX(days_since_prior_order) as max_days_since_prior_order,
  MIN(days_since_prior_order) as min_days_since_prior_order,
  AVG(order_dow) as avg_order_dow,
  MAX(order_dow) as max_order_dow,
  MIN(order_dow) as min_order_dow,
  max(reordered) max_reordered,
  sum(reordered) sum_reordered,
  avg(reordered) avg_reordered
FROM
  df_prior_tab
GROUP BY
  user_id, department_id""")

In [ ]:
## This table makes features for user bought products in consideration with cart


df_user_cart_30= sqlContext.sql("""SELECT
  user_id,
  count(1) as cnt_order_num,
  avg(cnt_order) as cnt_cart_num,
  avg(cnt_item) as avg_cnt_item,
  avg(cnt_aisle) as avg_cnt_aisle,
  avg(cnt_depart) as avg_cnt_depart,
  avg(sum_reordered) as avg_sum_reordered,
  sum(sum_reordered) as sum_reordered,
  avg(avg_reordered) as avg_reordered,
  avg(order_dow) as order_dow,
  avg(order_hour_of_day) as order_hour_of_day,
  avg(days_since_prior_order) as days_since_prior_order
FROM
(
SELECT
  a.user_id as user_id,
  a.order_id as order_id,
  count(1) cnt_order,
  COUNT(DISTINCT a.product_id) cnt_item,
  COUNT(DISTINCT a.aisle_id) cnt_aisle,
  COUNT(DISTINCT a.department_id) cnt_depart,
  sum(a.reordered) sum_reordered,
  avg(a.reordered) avg_reordered,
  avg(a.order_dow) as order_dow,
  avg(a.order_hour_of_day) as order_hour_of_day,
  avg(a.days_since_prior_order) as days_since_prior_order
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
GROUP BY
  a.user_id, a.order_id
)
GROUP BY
  user_id""")

In [ ]:
##   CASE WHEN NTH_VALUE(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH_VALUE(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and product id and then subtracting this from last buy value of current row 
## and last buy shouldnot be older than 30.

df_diff_user_item_30= sqlContext.sql("""SELECT
  user_id,
  product_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs

FROM
(
SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.product_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
) as s
GROUP BY
  user_id, product_id""")

In [ ]:
##CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and product id  and then subtracting this from last buy value of current row.

df_diff_user_item= sqlContext.sql("""SELECT
  user_id,
  product_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs  
FROM
(
SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.product_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
) as s
GROUP BY
  user_id, product_id""")

In [ ]:
###CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and aisle id and then subtracting this from last buy value of current row 
## and last buy shouldnot be older than 30.

df_diff_user_aisle_30= sqlContext.sql("""SELECT
  user_id,
  aisle_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs  
FROM
(
SELECT
  a.user_id as user_id,
  a.aisle_id as aisle_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.aisle_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
) as s
GROUP BY
  user_id, aisle_id""")


In [ ]:
##CASE WHEN NTH(51, QUANTILES(diffs - last_buy, 101)) is not NULL THEN NTH(51, QUANTILES(diffs - last_buy, 101)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and product id  and then subtracting this from last buy value of current row.

df_diff_user_aisle= sqlContext.sql("""SELECT
  user_id,
  aisle_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs  
FROM
(
SELECT
  a.user_id as user_id,
  a.aisle_id as aisle_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.aisle_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
) as s
GROUP BY
  user_id, aisle_id""")

In [ ]:
##CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and department id and then subtracting this from last buy value of current row 
## and last buy shouldnot be older than 30.


df_diff_user_depart_30= sqlContext.sql("""SELECT
  user_id,
  department_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs  
FROM
(
SELECT
  a.user_id as user_id,
  a.department_id as department_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.department_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
) as s
GROUP BY
  user_id, department_id""")


In [ ]:
##CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and department id  and then subtracting this from last buy value of current row.

df_diff_user_depart= sqlContext.sql("""SELECT
  user_id,
  department_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs  
FROM
(
SELECT
  a.user_id as user_id,
  a.department_id as department_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.department_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
) as s
GROUP BY
  user_id, department_id""")

In [ ]:
##CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and product id and then subtracting this from last buy value of current row 
## and last buy shouldnot be older than 30.


df_diff_user_30= sqlContext.sql("""SELECT
  user_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs  
FROM
(
SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.product_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
) as s
GROUP BY
  user_id""")


In [ ]:
##CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and product id  and then subtracting this from last buy value of current row.

df_diff_user= sqlContext.sql("""SELECT
  user_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs  
FROM
(
SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.product_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
) as s
GROUP BY
  user_id""")

In [ ]:
##CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and product id and then subtracting this from last buy value of current row 
## and last buy shouldnot be older than 30.


df_diff_item_30= sqlContext.sql("""SELECT
  product_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs  
FROM
(
SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.product_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
) as s
GROUP BY
  product_id""")

In [ ]:
##CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and product id  and then subtracting this from last buy value of current row.

df_diff_item= sqlContext.sql("""SELECT
  product_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs  
FROM
(
SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.product_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
) as s
GROUP BY
  product_id""")


In [ ]:
##CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and aisle id and then subtracting this from last buy value of current row 
## and last buy shouldnot be older than 30.


df_diff_aisle_30= sqlContext.sql("""SELECT
  aisle_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
    CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs
FROM
(
SELECT
  a.user_id as user_id,
  a.aisle_id as aisle_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.aisle_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
) as s
GROUP BY
  aisle_id""")

In [ ]:
##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and department id and then subtracting this from last buy value of current row 
## and last buy shouldnot be older than 30.

df_diff_depart_30= sqlContext.sql("""SELECT
  department_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs
FROM
(
SELECT
  a.user_id as user_id,
  a.department_id as department_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.department_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30
) as s
GROUP BY
   department_id""")


In [ ]:
##CASE WHEN NTH(501, QUANTILES(diffs - last_buy, 1001)) is not NULL THEN NTH(501, QUANTILES(diffs - last_buy, 1001)) ELSE -1 END as med_diffs,

##This table gets stddev, avg, min and max by first getting lastbuy value from the previous rows 
## based on user and aisle id  and then subtracting this from last buy value of current row 
## and last buy shouldnot be older than 30 and item should be reordered.

df_diff_user_item_reordered_30= sqlContext.sql("""SELECT
  user_id,
  product_id,
  CASE WHEN STDDEV(diffs - last_buy) is not NULL THEN STDDEV(diffs - last_buy) ELSE -1 END as std_diffs,
  CASE WHEN avg(diffs - last_buy) is not NULL THEN avg(diffs - last_buy) ELSE -1 END as avg_diffs,
  CASE WHEN min(diffs - last_buy) is not NULL THEN min(diffs - last_buy) ELSE -1 END as min_diffs,
  CASE WHEN max(diffs - last_buy) is not NULL THEN max(diffs - last_buy) ELSE -1 END as max_diffs
FROM
(
SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  LAG(b.last_buy, 1) OVER (PARTITION BY a.user_id, a.product_id ORDER BY a.order_number) as diffs,
  b.last_buy as last_buy
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
WHERE
  b.last_buy <= 30 and reordered = 1
) as s
GROUP BY
  user_id, product_id""")

In [ ]:

## This table extracts avg of recent items reordered in 4 quarters of a month based on user 
## and product id

df_user_item_recent_reordered= sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.product_id as product_id,
  AVG(CASE WHEN b.last_buy <=7 THEN reordered ELSE 0 END) as ui_under7,
  AVG(CASE WHEN b.last_buy > 7 AND b.last_buy <= 14 THEN reordered ELSE 0 END) as ui_under14,
  AVG(CASE WHEN b.last_buy > 14 AND b.last_buy <= 21 THEN reordered ELSE 0 END) as ui_under21,
  AVG(CASE WHEN b.last_buy > 21 AND b.last_buy <= 28 THEN reordered ELSE 0 END) as ui_under28,
  AVG(CASE WHEN b.last_buy > 28 THEN reordered ELSE 0 END) as ui_over28
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
GROUP BY
  a.user_id, product_id""")

In [ ]:

## This table extracts avg of recent items reordered in 4 quarters of a month based on user 
## and department id

df_user_depart_recent_reordered= sqlContext.sql("""SELECT
  a.user_id as user_id,
  a.department_id as department_id,
  AVG(CASE WHEN b.last_buy <=7 THEN reordered ELSE 0 END) as ui_under7,
  AVG(CASE WHEN b.last_buy > 7 AND b.last_buy <= 14 THEN reordered ELSE 0 END) as ui_under14,
  AVG(CASE WHEN b.last_buy > 14 AND b.last_buy <= 21 THEN reordered ELSE 0 END) as ui_under21,
  AVG(CASE WHEN b.last_buy > 21 AND b.last_buy <= 28 THEN reordered ELSE 0 END) as ui_under28,
  AVG(CASE WHEN b.last_buy > 28 THEN reordered ELSE 0 END) as ui_over28
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
GROUP BY
  a.user_id, department_id""")

In [ ]:

## This table extracts avg of recent items reordered in 4 quarters of a month based on aisle id

df_aisle_recent_reordered= sqlContext.sql("""SELECT
  a.aisle_id as aisle_id,
  AVG(CASE WHEN b.last_buy <=7 THEN reordered ELSE 0 END) as ui_under7,
  AVG(CASE WHEN b.last_buy > 7 AND b.last_buy <= 14 THEN reordered ELSE 0 END) as ui_under14,
  AVG(CASE WHEN b.last_buy > 14 AND b.last_buy <= 21 THEN reordered ELSE 0 END) as ui_under21,
  AVG(CASE WHEN b.last_buy > 21 AND b.last_buy <= 28 THEN reordered ELSE 0 END) as ui_under28,
  AVG(CASE WHEN b.last_buy > 28 THEN reordered ELSE 0 END) as ui_over28
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
GROUP BY
  aisle_id""")

In [ ]:

## This table extracts avg of recent items reordered in 4 quarters of a month based on department id

df_depart_recent_reordered= sqlContext.sql("""SELECT
  a.department_id as department_id,
  AVG(CASE WHEN b.last_buy <=7 THEN reordered ELSE 0 END) as ui_under7,
  AVG(CASE WHEN b.last_buy > 7 AND b.last_buy <= 14 THEN reordered ELSE 0 END) as ui_under14,
  AVG(CASE WHEN b.last_buy > 14 AND b.last_buy <= 21 THEN reordered ELSE 0 END) as ui_under21,
  AVG(CASE WHEN b.last_buy > 21 AND b.last_buy <= 28 THEN reordered ELSE 0 END) as ui_under28,
  AVG(CASE WHEN b.last_buy > 28 THEN reordered ELSE 0 END) as ui_over28
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
GROUP BY
  department_id""")

In [ ]:

## This table extracts avg of recent items reordered in 4 quarters of a month based on product id


df_item_recent_reordered= sqlContext.sql("""SELECT
  a.product_id as product_id,
  AVG(CASE WHEN b.last_buy <=7 THEN reordered ELSE 0 END) as i_under7,
  AVG(CASE WHEN b.last_buy > 7 AND b.last_buy <= 14 THEN reordered ELSE 0 END) as i_under14,
  AVG(CASE WHEN b.last_buy > 14 AND b.last_buy <= 21 THEN reordered ELSE 0 END) as i_under21,
  AVG(CASE WHEN b.last_buy > 21 AND b.last_buy <= 28 THEN reordered ELSE 0 END) as i_under28,
  AVG(CASE WHEN b.last_buy > 28 THEN reordered ELSE 0 END) as i_over28
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
GROUP BY
  product_id""")

In [ ]:
## This table extracts avg of recent items reordered in 4 quarters of a month based on user id

df_user_recent_reordered= sqlContext.sql("""SELECT
  a.user_id as user_id,
  AVG(CASE WHEN b.last_buy <=7 THEN reordered ELSE 0 END) as u_under7,
  AVG(CASE WHEN b.last_buy > 7 AND b.last_buy <= 14 THEN reordered ELSE 0 END) as u_under14,
  AVG(CASE WHEN b.last_buy > 14 AND b.last_buy <= 21 THEN reordered ELSE 0 END) as u_under21,
  AVG(CASE WHEN b.last_buy > 21 AND b.last_buy <= 28 THEN reordered ELSE 0 END) as u_under28,
  AVG(CASE WHEN b.last_buy > 28 THEN reordered ELSE 0 END) as u_over28
FROM
  df_prior_tab as a
LEFT OUTER JOIN
  df_cum_orders as b
ON
  a.order_id = b.order_id
GROUP BY
  a.user_id""")

In [ ]:
df_only_rebuy_train.registerTempTable("df_only_rebuy_train")
df_only_rebuy_test.registerTempTable("df_only_rebuy_test")
df_last_buy.registerTempTable("df_last_buy")
df_last_buy_aisle.registerTempTable("df_last_buy_aisle")
df_last_buy_depart.registerTempTable("df_last_buy_depart")
df_last_buy_2.registerTempTable("df_last_buy_2")
df_last_buy_aisle_2.registerTempTable("df_last_buy_aisle_2")
df_last_buy_depart_2.registerTempTable("df_last_buy_depart_2")
df_dmt_user_item_30.registerTempTable("df_dmt_user_item_30")
df_dmt_user_aisle_30.registerTempTable("df_dmt_user_aisle_30")
df_dmt_user_depart_30.registerTempTable("df_dmt_user_depart_30")
df_dmt_user_item.registerTempTable("df_dmt_user_item")
df_dmt_user_aisle.registerTempTable("df_dmt_user_aisle")
df_dmt_user_depart.registerTempTable("df_dmt_user_depart")
df_user_cart_30.registerTempTable("df_user_cart_30")
df_diff_user_item_30.registerTempTable("df_diff_user_item_30")
df_diff_user_item.registerTempTable("df_diff_user_item")
df_diff_user_aisle_30.registerTempTable("df_diff_user_aisle_30")
df_diff_user_aisle.registerTempTable("df_diff_user_aisle")
df_diff_user_depart_30.registerTempTable("df_diff_user_depart_30")
df_diff_user_depart.registerTempTable("df_diff_user_depart")
df_diff_user_30.registerTempTable("df_diff_user_30")
df_diff_user.registerTempTable("df_diff_user")
df_diff_item_30.registerTempTable("df_diff_item_30")
df_diff_item.registerTempTable("df_diff_item")
df_diff_aisle_30.registerTempTable("df_diff_aisle_30")
df_diff_depart_30.registerTempTable("df_diff_depart_30")
df_diff_user_item_reordered_30.registerTempTable("df_diff_user_item_reordered_30")
df_user_item_recent_reordered.registerTempTable("df_user_item_recent_reordered")
df_user_depart_recent_reordered.registerTempTable("df_user_depart_recent_reordered")
df_aisle_recent_reordered.registerTempTable("df_aisle_recent_reordered")
df_depart_recent_reordered.registerTempTable("df_depart_recent_reordered")
df_item_recent_reordered.registerTempTable("df_item_recent_reordered")
df_user_recent_reordered.registerTempTable("df_user_recent_reordered")

##TABLES NOT PRESENT IN SQL

# ##LEFT OUTER JOIN
#   [instacart.dmt_item2_30] as i2
# ON
#   i2.i1_product_id = o.product_id

# LEFT OUTER JOIN
#   [instacart.diff_user_aisle_reordered_30] as dai3
# ON
#   dai3.user_id = o.user_id AND p.aisle_id = dai3.aisle_id

# LEFT OUTER JOIN
#   [instacart.diff_user_depart_reordered_30] as dddi3
# ON
#   dddi3.user_id = o.user_id AND  p.department_id = dddi3.department_id
    
# LEFT OUTER JOIN
#   [instacart.dmt_user2_30] as u2
# ON
#   u2.u1_user_id = o.user_id

# LEFT JOIN [instacart.dmt_item2_30] as i2 ON i2.i1_product_id = o.product_id in second

# i2.*,dai3.*,dddi3.*,u2.*,i2.*







In [ ]:
## This is used to make training dataset for problem under consideration

df_dmt_train_only_rebuy= sqlContext.sql("""SELECT
  CASE WHEN tr.reordered is not null THEN tr.reordered ELSE 0 END as target,
  o.user_id,
  p.aisle_id,
  p.department_id,
  o.product_id,
  o.order_id,
  o.order_number,
  o.order_dow,
  o.order_hour_of_day,
  o.days_since_prior_order,
  o.cum_days,
  du.*,
  dui3.*,
  dd.*,
  du3.*,
  da3.*,
  dd3.*,
  udd.*,
  udd3.*,
  di.*,
  di3.*,
  ddi3.*,
  ddd3.*,
  u.*,
  uc.*,
  i.*, 
  l.*,
  la.*,
  ld.*,
  l2.*,
  la2.*,
  ld2.*,
  ui.*,
  ua.*,
  ud.*,
  ui3.*,
  ua3.*,
  ud3.*,
  rui.*,
  ru.*,
  ri.*
FROM
  df_only_rebuy_test as o LEFT JOIN products_df as p ON o.product_id = p.product_id
  LEFT JOIN df_diff_user_item as du ON du.user_id = o.user_id AND  o.product_id = du.product_id
  LEFT JOIN df_diff_user_item_reordered_30 as dui3 ON dui3.user_id = o.user_id AND  o.product_id = dui3.product_id
  LEFT JOIN df_diff_user_depart as dd ON dd.user_id = o.user_id AND  p.product_id = dd.department_id
  LEFT JOIN df_diff_user_item_30 as du3 ON du3.user_id = o.user_id AND  o.product_id = du3.product_id
  LEFT JOIN  df_diff_user_aisle_30 as da3 ON da3.user_id = o.user_id AND  p.aisle_id = da3.aisle_id
  LEFT JOIN df_diff_user_depart_30 as dd3 ON dd3.user_id = o.user_id AND p.product_id = dd3.department_id
  LEFT JOIN df_diff_user as udd ON udd.user_id = o.user_id
  LEFT JOIN df_diff_user_30 as udd3 ON udd3.user_id = o.user_id
  LEFT JOIN df_diff_item as di ON di.product_id = o.product_id
  LEFT JOIN df_diff_item_30 as di3 ON di3.product_id = o.product_id
  LEFT JOIN df_diff_aisle_30 as ddi3 ON ddi3.aisle_id = p.aisle_id
  LEFT JOIN df_diff_depart_30 as ddd3 ON ddd3.department_id = p.department_id
  LEFT JOIN df_dmt_user as u ON u.user_id = o.user_id
  LEFT JOIN df_user_cart_30 as uc ON uc.user_id = o.user_id
  LEFT JOIN df_dmt_item as i ON i.product_id = o.product_id
  LEFT JOIN df_last_buy as l ON l.user_id = o.user_id AND l.product_id = o.product_id
  LEFT JOIN df_last_buy_aisle as la ON la.user_id = o.user_id AND la.aisle_id = p.aisle_id
  LEFT JOIN df_last_buy_depart as ld ON ld.user_id = o.user_id AND ld.department_id = p.department_id
  LEFT JOIN df_last_buy_2 as l2 ON l2.user_id = o.user_id AND l2.product_id = o.product_id
  LEFT JOIN df_last_buy_aisle_2 as la2 ON la2.user_id = o.user_id AND la2.aisle_id = p.aisle_id
  LEFT JOIN df_last_buy_depart_2 as ld2 ON ld2.user_id = o.user_id AND ld2.department_id = p.department_id
  LEFT JOIN df_dmt_user_item as ui ON ui.user_id = o.user_id AND ui.product_id = o.product_id
  LEFT JOIN df_dmt_user_aisle as ua ON ua.user_id = o.user_id  AND ua.aisle_id = p.aisle_id
  LEFT JOIN df_dmt_user_depart as ud ON ud.user_id = o.user_id AND ud.department_id = p.department_id
  LEFT JOIN df_dmt_user_item_30 as ui3 ON ui3.user_id = o.user_id AND ui3.product_id = o.product_id
  LEFT JOIN df_dmt_user_aisle_30 as ua3 ON ua3.user_id = o.user_id AND ua3.aisle_id = p.aisle_id
  LEFT JOIN df_dmt_user_depart_30 as ud3 ON ud3.user_id = o.user_id AND ud3.department_id = p.department_id
  LEFT JOIN df_user_item_recent_reordered as rui ON rui.user_id = o.user_id AND rui.product_id = o.product_id
  LEFT JOIN df_user_recent_reordered as ru ON ru.user_id = o.user_id
  LEFT JOIN df_item_recent_reordered as ri ON ri.product_id = o.product_id
  LEFT JOIN df_train_tab as tr ON tr.user_id = o.user_id AND tr.product_id = o.product_id AND tr.order_id = o.order_id

""")

In [ ]:
##  LEFT OUTER JOIN [instacart.diff_user_aisle_reordered_30] as dai3 ON dai3.user_id = o.user_id AND p.aisle_id = dai3.aisle_id
## LEFT JOIN [instacart.diff_user_depart_reordered_30] as dddi3 ON dddi3.user_id = o.user_id AND p.department_id = dddi3.department_id

## This is used to make testing dataset for problem under consideration

df_dmt_test_only_rebuy= sqlContext.sql("""
SELECT
  o.user_id,
  p.aisle_id,
  p.department_id,
  o.product_id,
  o.order_id,
  o.order_number,
  o.order_dow,
  o.order_hour_of_day,
  o.days_since_prior_order,
  o.cum_days,
  du.*,
  dui3.*,
  dd.*,
  du3.*,
  da3.*,
  dd3.*,
  udd.*,
  udd3.*,
  di.*,
  di3.*,
  ddi3.*,
  ddd3.*,
  u.*,
  uc.*,
  i.*,
  l.*,
  la.*,
  ld.*,
  l2.*,
  la2.*,
  ld2.*,
  ui.*,
  ua.*,
  ud.*,
  ui3.*,
  ua3.*,
  ud3.*,
  rui.*,
  ru.*,
  ri.*
FROM
  df_only_rebuy_test as o LEFT JOIN products_df as p ON o.product_id = p.product_id
  LEFT JOIN df_diff_user_item as du ON du.user_id = o.user_id AND o.product_id = du.product_id
  LEFT JOIN df_diff_user_item_reordered_30 as dui3 ON dui3.user_id = o.user_id AND o.product_id = dui3.product_id
  LEFT JOIN df_diff_user_depart as dd ON dd.user_id = o.user_id AND p.product_id = dd.department_id
  LEFT JOIN df_diff_user_item_30 as du3 ON du3.user_id = o.user_id AND o.product_id = du3.product_id
  LEFT JOIN df_diff_user_aisle_30 as da3 ON da3.user_id = o.user_id AND p.aisle_id = da3.aisle_id
  LEFT JOIN df_diff_user_depart_30 as dd3 ON dd3.user_id = o.user_id AND p.product_id = dd3.department_id
  LEFT JOIN df_diff_user as udd ON udd.user_id = o.user_id
  LEFT JOIN df_diff_user_30 as udd3 ON udd3.user_id = o.user_id
  LEFT JOIN df_diff_item as di ON di.product_id = o.product_id
  LEFT JOIN df_diff_item_30 as di3 ON di3.product_id = o.product_id
  LEFT JOIN df_diff_aisle_30 as ddi3 ON ddi3.aisle_id = p.aisle_id
  LEFT JOIN df_diff_depart_30 as ddd3 ON ddd3.department_id = p.department_id
  LEFT JOIN df_dmt_user as u ON u.user_id = o.user_id
  LEFT JOIN df_user_cart_30 as uc ON uc.user_id = o.user_id
  LEFT JOIN df_dmt_item as i ON i.product_id = o.product_id
  LEFT JOIN df_last_buy as l ON l.user_id = o.user_id AND l.product_id = o.product_id  
  LEFT JOIN df_last_buy_aisle as la ON la.user_id = o.user_id AND la.aisle_id = p.aisle_id
  LEFT JOIN df_last_buy_depart as ld ON ld.user_id = o.user_id AND ld.department_id = p.department_id
  LEFT JOIN df_last_buy_2 as l2 ON l2.user_id = o.user_id AND l2.product_id = o.product_id
  LEFT JOIN df_last_buy_aisle_2 as la2 ON la2.user_id = o.user_id AND la2.aisle_id = p.aisle_id
  LEFT JOIN df_last_buy_depart_2 as ld2 ON ld2.user_id = o.user_id AND ld2.department_id = p.department_id
  LEFT JOIN df_dmt_user_item as ui ON ui.user_id = o.user_id AND ui.product_id = o.product_id
  LEFT JOIN df_dmt_user_aisle as ua ON ua.user_id = o.user_id AND ua.aisle_id = p.aisle_id
  LEFT JOIN df_dmt_user_depart as ud ON ud.user_id = o.user_id AND ud.department_id = p.department_id
  LEFT JOIN df_dmt_user_item_30 as ui3 ON ui3.user_id = o.user_id AND ui3.product_id = o.product_id
  LEFT JOIN df_dmt_user_aisle_30 as ua3 ON ua3.user_id = o.user_id AND ua3.aisle_id = p.aisle_id
  LEFT JOIN df_dmt_user_depart_30 as ud3 ON ud3.user_id = o.user_id AND ud3.department_id = p.department_id
  LEFT JOIN df_train_tab as tr ON tr.user_id = o.user_id AND tr.product_id = o.product_id AND tr.order_id = o.order_id
  LEFT JOIN df_user_item_recent_reordered as rui ON rui.user_id = o.user_id AND rui.product_id = o.product_id
  LEFT JOIN df_user_recent_reordered as ru ON ru.user_id = o.user_id
  LEFT JOIN df_item_recent_reordered as ri ON ri.product_id = o.product_id
""")